## 12.1 文本生成

这一节中讨论的是利用 rnn 生成序列数据,以文本生成为例,但是相同的技术可以推广到任何种类的序列数据上.音乐/作画等等.

序列数据不仅限于艺术内容生成,目前已经被成功应用在了语音合成与聊天机器人.google 在 2016 年的智能回复功能中就能实现电子邮件/短信息的快速回复,其背后的技术是同源的.


## 简史

2014 年以前很少见到 LTSM 的字样,即使在机器学习领域.使用 rnn 生成序列数据基本上要在 2016 年以后才出现在主流领域.但这些技术本身有相当长的历史,ltsm 在 1997 年为了逐个生成文本而开发.

2002 年在瑞士 Schmidhuber 实验室工作的 Douglas Eck 首次将 ltsm 应用在了音乐生成上,并取得了令人满意的结果.Eck 现在是 Google 大脑的研究人员,2016 年创建了名为 Magenta 的小组,致力于使用深度学习技术创造更加迷人的音乐.有时候,好的想法需要 15 年才能变为实践.

2000 ~ 2010年初,Alex Graves 使用 rnn 在序列生成方面做了开创性工作,特别是 2013 年使用循环混合密度网络用于生成类似人类的手写笔迹,被人看作是这一领域的转折点.在那个时刻,笔迹生成这一个特定的银行业,能做梦的机器这一概念适时引起了我的星期,并在我开发 keras 是提供了重要的灵感.Graves 在 2013 年上传预印本到 arXiv 时在 latex 文件留下了一段类似注释的评论: 序列数据生成是计算机所能做的最接近做梦的事情.几年后我们认为这一切的发展都理所当然,但在当时看到了 Graves 的演示,很难不为其中蕴藏的可能性而震撼.在 2015~2017 年,rnn 被成功应用在了文本和对话生成,音乐合成,语音合成等领域.

然后在 2017~2018 年 Transformer 架构开始替代 rnn,不仅仅是在有监督的 nlp 任务上,在序列生成(特别是语言建模/单词级别文本生成)也开始替换 rnn.最著名的 Transformer 生成式应用可能是 gpt-3,这个模型包含了 1750 亿个参数的文本生成模型.由 openai 在一个令人震惊的数据集上训练,这个数据集囊括了大部分可用的数字书籍 维基百科 和整个互联网的很大一部分抓取内容.gpt-3 在 2020 年成了头条新闻,因为其有能力就任何话题生成一段看起来合理的文本,这种能力引发了短暂的炒作热潮.

---

下面是我加的内容;

gpt-3 的论文中有一句话,我们发现了一个 bug 但是因为训练成本,无法解决...有人保守估计 gpt-3 的训练费用在数百万~数千万美金之间,真的是模型每个字节按照美元算..

在没有算法/模型的重大突破前,如何有一个通用性最好的模型呢?--将全部的人类文本资料全部扔到模型训练,并且过拟合,大力出奇迹,力大砖飞... gpt-3 就是目前最接近这样的通用模型.

2021 年 github 推出了 copilot,辅助程序开发,怎么说呢,现有的深度学习终将替换掉一切重复劳动的工作.


## 如何生成序列数据?

深度学习产生序列数据的通用方法:

- 训练一个预测模型(通常是 rnn 或者 Transformer)
- 使用之前的标记作为输入,通过模型得到下一个/下几个标记.
- 重复上面的步骤直到结束.

例如给定输入 'the cat is on the',模型被用来预测输出下一个标记 'mat'.

标记通常指的是单词或者字符,~~能够根据之前标记输出下一个标记的模型都被称作语言模型.~~(查了下概念,这句话翻译有歧义).

一旦有了这样一个训练有素的语言模型后,我们就能开始取样了(生成新序列):

- 提供初始文本(又被称为条件数据)
- 要求模型输出下一个标记
- 将输出的标记+之前的文本再重新输入模型

以上过程可重复 N 多次,这个循环允许你生成任意长度的序列,最终得到的序列看起来就像人类书写的句子.

![text_generation_process](text_generation_process.png)


## 采样策略

生成文本时,如何选择下一个符号非常重要,一种简单的方法是贪婪采样: 每次只选择可能性最大的下一个字符.这样做的后果是最终的序列成为了重复可预测字符串,而不是连贯的语言.

一种更加有趣的方式是在采样中引入随机性,即从下一个字符的概率分布中进行采样--这样叫做随机采样.这种情况下,根据模型的结果,如果下一个字符是 e 的可能性是 0.3,那最终选择 e 的可能性就是 30%.贪婪采样也能看作是一种随机采样,只是某个字符概率是 1,其他都是 0.

从模型的 softmax 输出中进行概率采样是一种很巧妙的办法,它甚至可用采样到不常见字符,从而使得生成更加有趣的句子.而且有时又会得到完全不再样本的新单词,表现一定的创造性.但是随机取样的问题是: 随机取样无法控制随机性的大小.

又有另一个问题,为什么要具有随机性?

- 考虑一个极端例子,纯随机采样.选取下一个字符每个字符概率均等.这样得到的序列最大的随机性,换句话说这样的序列有最大的熵.当然完全随机的序列不会有任何有趣的内容.
- 另一个极端例子,贪婪采样.最终得到的序列不会有任何的随机性,同样不会有任何有趣的内容.对应的熵是最小的.
- 更小的熵会让序列有可预测的结构(看起来更加真实),而更大的熵会得到根据创造性的序列.
- 两个例子中间有着相当广阔的空间,我们需要的是这中间的某个点.因为人是最终序列的判定者,人是主观的,有趣的标准因人而异,因此我们无法提前获知最佳熵的位置.

为了控制采样过程中的随机性,我们引入一个叫做 softmax 温度的参数.用来表示采样概率分布的熵,即所选择的下一个字符有多么出人意料或者多么不可预测.给定一个 temperature 值,将按照下面的方法对原始概率分布进行重新加权,计算得到一个新的概率分布.


In [1]:
import numpy as np


def reweight_distribution(original_distribution, temperature=0.5):
    """[summary]

    Args:
        original_distribution ([type]): [概率分布的一维 numpy 数组概率之和必须为 1]
        temperature (float, optional): [一个因子,定量描述输出的分布熵]. Defaults to 0.5.

    Returns:
        [type]: [description]
    """
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(
        distribution
    )  #返回原始分布重新加权后结果,重新加权后 distribution 求和可能不再为 1,因此需要除以求和得到新的分布


![temperature](temperature.png)

更高的温度对应的是熵更大的采样分布,更低的温度对应的是更小的随机性.


## 使用 keras 编写文本生成

让我们使用 keras 完成文本生成.为了训练模型,首先需要大量的文本数据,可以是任何足够大的文本文件或者文本数据集(指环王,维基百科等等).

这个例子中我们使用上一章的 imdb 数据集,对应的我们的模型将是专门生成针对这些电影评论的模型,而不是一般的英文语言模型.


### 准备数据


In [2]:
# !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

最终得到的是 aclImdb 文件夹,包含两个子文件夹,一个负面 一个正面.

每个评论有一个文本文件。我们将调用 text_dataset_from_directory，标签模式=None,创建一个数据集,从这些文件中读取,并得到每个文件的文本内容.


In [3]:
import tensorflow as tf
from tensorflow import keras

dataset = keras.preprocessing.text_dataset_from_directory(directory="aclImdb",
                                                          label_mode=None,
                                                          batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " ")
                      )  #剔除 '<br />',<br /> 是 html 的标签,但是文本生成并不关心


Found 100006 files belonging to 1 classes.


In [4]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

sequence_length = 100  #每个句子的长度
vocab_size = 15000  #词典的大小
# 最终只有 15000 个常见词

text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",  # 单词索引序列 int 类型
    output_sequence_length=sequence_length,  #长度 100.但因为偏移 1,最终使用的是 99 长度 
)
text_vectorization.adapt(dataset)

我们使用 TextVectorization 层计算我们需要的词汇,每篇评论只使用 sequence_length 以内的词.TextVectorization 层进行文本矢量化时会将超过 sequence_length 的词截断.


In [5]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)  #字符串转成 int 序列
    x = vectorized_sequences[:, :-1]  #截断序列最后一个 创建输入
    y = vectorized_sequences[:, 1:]  #序列偏移1 创建目标序列
    return x, y


lm_dataset = dataset.map(prepare_lm_dataset)
lm_dataset = lm_dataset.prefetch(8)  #更好性能,确保每次内存都有 8 个批次预取.


### 基于 Transformer 序列到序列模型

我们在这一个小节将会训练一个模型,提供会模型一个词会返回句子中下一个词的概率分布.模型训练完毕,我们提供初始的文本,不断进行采样,直到得到我们需要的文本规模.

与第 10 章温度预测的工作类似,我们会训练一个模型,将 N 个词作为输入预测出 N+1 个词,然而这个过程无法在文本生成直接套用.

- 这样的模型只能在有 N 个词的时候进行预测,当 N 比较大时,我们被限制在需要提供比较大的初始输入.
- 这样训练的大部分序列将是重叠的,以 N = 4 时为例,'A complete sentence must have, at minimum, three things: a subject, verb, and an object' 拆分
  - "A complete sentence must"
  - "complete sentence must have"
  - "sentence must have at"
  - 等等直到 "verb and an object".
  
  每一个这样的序列视为独立样本,模型将不得不做大量的冗余工作.对源序列的处理阶段将不得不重复编码大量相似但不相同的子序列.
  - 有一种办法是,在取两个子序列时,一次跨过几个单词,但这样会减少训练的样本量,同时也只是部分解决问题,没有根本解决.


![seq2seq0001](seq2seq0001.png)

另外一种彻底的解决办法:

- N 个词的序列,一次送入模型,模型内部将输入序列偏移 1.
- 采取一些屏蔽策略,确保每一个 i ($i\in [0,N]$) 模型都只使用输入序列 $[0,i]$ 范围内的单词预测 i+1 个词.

这样解决了上面的两个问题,同时模型哪怕只给 1 个初始单词,也能给出下一个词的概率分布.

这样的解决方式也能在第 10 章的任务上使用,第 10 章温度预测问题上,我们要给定一个 120h 的温度序列,然后学习生成一个 120h 的温度序列,将偏移设置为未来 24h.这样不但解决了最初的问题,还解决了额外的 199 个相关问题.问题发生了变化:

- 原问题: 给定 120h 的温度数据,预测接下来 24h 温度
- 变换后问题: 给定少于 120h 的温度数据,预测接下来 24h 问题

同样的训练后,结果可能会类似,但是效果会变差,因为这样的模型除了原问题还关心了额外的 199 个相似但不完全相同的问题,这样会轻微干扰原问题的解决.


上一章我们已经了解了序列到序列的模型学习

- 源序列送入编码器
- 编码器将源序列的编码输出 + 目标序列 送入解码器.
- 解码器试图预测偏移以后 目标序列的下一个标记.

上面的流程在做文本生成时需要调整.

文本生成没有源序列: 我们只是试图预测目标序列的下一个标记,这个过程我们只能使用解码器.由于因果填充,解码器只能看到 0~N 来预测 N+1 个单词.


In [7]:
import tensorflow.keras.layers as layers

In [8]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim,
                 **kwargs):  #位置嵌入缺点是必须事先知道序列长度
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim,
                                                 output_dim=output_dim)  #嵌入层
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)  #标记位置嵌入层
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions  #两个嵌入层相加

    def compute_mask(
            self,
            inputs,
            mask=None):  #与直接使用 Embedding 一样,这个层应该能够生成 mask,这样就能忽略掉填充的数据,减少运算量.
        return tf.math.not_equal(inputs, 0)

    def get_config(self):  #保存模型时调用
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [9]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True  #这个属性确保了该层将输入序列的 mask 传播到输出上.

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(
            i >= j,
            dtype="int32")  #产生形状 （sequence_length, sequence_length）的 0-1 矩阵
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32)
            ],
            axis=0
        )  #沿着批次轴复制它，得到一个形状矩阵（batch_size, sequence_length, sequence_length）。
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)  #获取 causal_mask
        if mask is not None:  #准备输入掩码
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)  #合并掩码
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs,
            attention_mask=causal_mask)  # 因果掩码传递给 self-attention
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,  #将合并后的掩码传递给第二注意力层，该层将源序列与目标序列联系起来。
        )
        attention_output_2 = self.layernorm_2(attention_output_1 +
                                              attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


这里要重用 11 章的 PositionalEmbedding 和 TransformerDecoder.


In [10]:
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None, ), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(
    x)  #对可能的词汇进行softmax,对每一个输出序列的步长进行计算.
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

## 可变温度采样回调的文本生成

我们在这一节会创建一个带回调函数的文本生成模型.在每个迭代后使用不同的 temperature 生成文本.这能让我们看到随着模型收敛,生成文本是如何演变的.以及不同的温度会如何影响文本生成.

这一节我们使用的初始种子文本将始终是 'this movie'.


In [11]:
import numpy as np

tokens_index = dict(enumerate(
    text_vectorization.get_vocabulary()))  # 单词索引映射回字符串,文本解码.


def sample_next(predictions, temperature=1.0):  #随机采样 变温的
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)


class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1., ),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt  #开始句子
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])  #当前序列送入模型
                predictions = self.model(tokenized_sentence)  #获取预测
                next_token = sample_next(predictions[0, i, :])  #检索预测结果
                sampled_token = tokens_index[next_token]  #
                sentence += " " + sampled_token  #新词加入序列
            print(sentence)


prompt = "This movie"
text_gen_callback = TextGenerator(prompt,
                                  generate_length=50,
                                  model_input_length=sequence_length,
                                  temperatures=(0.2, 0.5, 0.7, 1.,
                                                1.5))  #使用不同的温度进行采样


In [13]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
391/391 [==============================] - 152s 376ms/step - loss: 5.3558
== Generating with temperature 0.2
This movie is has horrible at animation least sex but at it the is end just when not one something is i the will twilight enjoy zone and today didnt overall mean for sometimes beauty this is is at the all worst the performance worst not movie even where better there throughout
== Generating with temperature 0.5
This movie is has a such very an well attempt acted to comedy even or come thrill and its sees just that a shows film what after with another fullest [UNK] [UNK] builds to up understand in how a more breath then of when it its weve norwegian got yorker loose save
== Generating with temperature 0.7
This movie a was very a idea good of actor the especially cast liked conan it william excellent mccarthy acting this and in it her from party acting nor outside character them too did at a the kills students filmmakers jack just but imagine the this director film to 

KeyboardInterrupt: 

运行时间实在是太长了,不得不中断,这里取的是原文的结果.因为标点符号被剔除了,因此结果不含标点.

With temperature=0.2:

- "this movie is a [UNK] of the original movie and the first half hour of the movie is pretty good but it is a very good movie it is a good movie for the time period"
- "this movie is a [UNK] of the movie it is a movie that is so bad that it is a [UNK] movie it is a movie that is so bad that it makes you laugh and cry at the same time it is not a movie i dont think ive ever seen"

With temperature=0.5:

- "this movie is a [UNK] of the best genre movies of all time and it is not a good movie it is the only good thing about this movie i have seen it for the first time and i still remember it being a [UNK] movie i saw a lot of years"
- "this movie is a waste of time and money i have to say that this movie was a complete waste of time i was surprised to see that the movie was made up of a good movie and the movie was not very good but it was a waste of time and"

With temperature=0.7:

- "this movie is fun to watch and it is really funny to watch all the characters are extremely hilarious also the cat is a bit like a [UNK] [UNK] and a hat [UNK] the rules of the movie can be told in another scene saves it from being in the back of"
- "this movie is about [UNK] and a couple of young people up on a small boat in the middle of nowhere one might find themselves being exposed to a [UNK] dentist they are killed by [UNK] i was a huge fan of the book and i havent seen the original so it"

With temperature=1.0:

- "this movie was entertaining i felt the plot line was loud and touching but on a whole watch a stark contrast to the artistic of the original we watched the original version of england however whereas arc was a bit of a little too ordinary the [UNK] were the present parent [UNK]"
- "this movie was a masterpiece away from the storyline but this movie was simply exciting and frustrating it really entertains friends like this the actors in this movie try to go straight from the sub thats image and they make it a really good tv show"

With temperature=1.5:

- "this movie was possibly the worst film about that 80 women its as weird insightful actors like barker movies but in great buddies yes no decorated shield even [UNK] land dinosaur ralph ian was must make a play happened falls after miscast [UNK] bach not really not wrestlemania seriously sam didnt exist"
- "this movie could be so unbelievably lucas himself bringing our country wildly funny things has is for the garish serious and strong performances colin writing more detailed dominated but before and that images gears burning the plate patriotism we you expected dyan bosses devotion to must do your own duty and another"


temperature 越低,文本越重复无聊.0.2° 时还有很多重复文本.随着 temperature 升高,文本的创造性越来越好,开始变的有趣,当时最终 temperature = 1.5 时,生成序列开始变得不成章法,无法阅读.按照结果来看,一个平衡点是 0.7°.

要注意的是,通过更多的数据,更长的时间,更大的模型,我们是可以获得更加通畅/有趣/真实的生成文本的.gpt-3 就是一个例子.但是不能指望这样能够产生真正有意义的文本,除了随机的抽取文字和人类对文本的解释,这样的过程实际上是从一个统计模型中抽取数据,语言模型只有形式没有真正的内容.


自然语言在人类世界中可以是很多东西: 一种沟通的媒介,一种感染世界的行动方式,一种社会的润滑剂,一种存储/表达你自己想法的方式.语言的这些实际用途是语言起源的意义.一个深度学习的 '语言模型',尽管名字带了语言俩字,也仅仅是因为样本来自人类语言.这样的模型无法交流(没有交流的必要,也没有交流的对象),不能对现实世界采取什么行动(没有意图),不能称为一个社会意义上的人,没有任何要表达的思想.语言是心灵的操作系统,要创造真正有意义的语言,需要一个真正意义上的心灵作为输出方.

语言模型所作的捕捉人类活动所产生文本(书籍,电影评论,推特)的模式.这些文本存在统计意义上的结构是人类实现语言的一个副产物.有一个思想实验: 假如人与人之间的沟通语言也像计算机之间通信一样,没有那么多的冗余,再做到更好的压缩.语言原本的目的还是一样实现,但是这样的文本不再有任何有意义统计意义上的结构,自然也无法进行深度学习.

---

题外话:

在目前的观点中,似乎都将语言和心灵/智能捆绑在了一起,但是假设一个类似的模型,训练过全部的人类互联网文字,这样的模型非常可能通过在线的图灵测试,那这样的模型算是有智能吗?

类似的话题是中文房间,哲学僵尸等等.现在我们对智能的定义还在初级阶段,随着对大脑和人工智能发展,或许在有生之年能看到一丝曙光.

---


## 一点收尾

我们可以训练一个序列到序列的模型,从给定以前的标记,预测下一个标记.

在使用文本数据情况下,这样的模型称为语言模型,单个标记可以是字符或单词.

对下一个标记的采样要在规律性和随机性之间取得平衡.

处理随机性问题一个方法是引入 softmax 温度,始终使用不同的温度尝试,找到一个平衡点.
